In [14]:
# HIDDEN

import sys

import os

sys.path.append(os.path.join(os.path.abspath(''), '..'))

Entity Component Systems are an excellent solution for maintaining data separate from behavior in a rendering engine. This separation allows for better parallelization in the running code. Throughout this notebook, we present the translation of runtime data to a graph format, where the relationships between entities and components are visualized and well-suited for querying. We provide a simple example where each type of component is stored is a graph database along its relationship with entities. We read the type using reflection and typing mechanisms, and serialize it with the Cipher language into Neo4j.

In [15]:
import ECS.ecsWorld as ECS

In [16]:
from engine.ECS.hierarchy import Hierarchy
from engine.ECS.lifetime import Lifetime
from engine.ECS.physics.rigidbody import Rigidbody
from engine.ECS.render.geometryRenderer import GeometryRenderer
from engine.ECS.transform import Transform

In this example, we create a particle entity and attach a variety of components related to its transformation, rendering, and physics. As demonstrated, the data is composed in a relational manner, rather than the more traditional approach of decorating an object. Here, the entity is simply an ID, and a 'table' maintains the relationships to each component type. To introduce complexity and different relationships, we add a transform hierarchy to model parent-child relationships. In the ECS, the transformation service computes the global transform for each object based on its transform and hierarchy components. The hierarchy component is the means of adding parent information to an entity and contains a reference to another entity.

In the following example, we model a parent entity with a simple transformation component and a child particle with additional behavior attached. Our primary interest is in reproducing the data relationships into an equivalent graph, enabling us to ask simpler questions about the scene. Some of the properties within the components are initialized to None, as the purpose of this example is to demonstrate serialization and not all objects are necessary (the Geometry Renderer requires dependencies from the rendering engine).

In [17]:
particle_parent = ECS.create_entity()

In [18]:
# parent components

ECS.add_component(particle_parent, Transform(scale=np.array([1.5, 1.5, 1.5])))

In [19]:

particle_instance = ECS.create_entity()

In [20]:
# child components

ECS.add_component(particle_instance, Hierarchy(parent=particle_parent))
ECS.add_component(particle_instance, Transform(scale=np.array([0.5, 0.5, 0.5])))
ECS.add_component(particle_instance, GeometryRenderer(None, None))
ECS.add_component(particle_instance, Rigidbody(mass = 1.0))
ECS.add_component(particle_instance, Lifetime(life = 0.5))

We aim to translate this information into a graph-like structure and later visualize it. Additionally, a new query method allows for asking scene graphs in a different, less relational way. We want to maintain the relational approach for storage and processing in systems, but the graph transformation enables a more connected way to retrieve information about a possible snapshot of a scene.

In [21]:
# driver, entity_id, are common for all linking functions;

def link_hierarchy(driver, entity_id, parent_id):

    driver.execute_query(f"""

        MATCH
            (chd_{entity_id}:Entity {{ name : "{entity_id}" }} ),
            (prt_{parent_id}:Entity {{ name : "{parent_id}" }} )
        CREATE 
            (chd_{entity_id})-[:IS_CHILD  {{role: 'family' }}]->(prt_{parent_id}),
            (prt_{parent_id})-[:IS_PARENT {{role: 'family' }}]->(chd_{entity_id})
                                                
    """)

In [22]:
dict_t = { 
    
    "Hierarchy" : (link_hierarchy, "parent") 
}

In order to serialize the components, we need to read their type dynamically. For each component, the class type is translated into a graph node type, and each attribute is read and appended to the node to maintain the same data. Since each entity has a list of components, we model this relationship in the graph.

There may be many other types or relationships between components and entities, but for this example, we have decided to translate the hierarchy component into the graph. We want to keep track of which entity has what parent and vice versa. For this process, a little extra code models the relationship when the 'Hierarchy' component is found in the list of entity components. It looks for the parent reference stored in the hierarchy and later translates it into the graph by linking the nodes with a bidirectional relationship: IS_PARENT, IS_CHILD.

In [23]:
def serialize_entity(driver, ent):

    entity_id = ent

    driver.execute_query(f"""
                                                    
            CREATE(entity_{entity_id}:Entity {{ name : "{entity_id}" }} )
                                                    
    """)

    for component in ECS.components_for_entity(ent):

        """ store variables with their state in each node component """

        cmp_nme = component.__class__.__qualname__

        dic_str = str({key: '"'+str(val)+'"' for key, val in component.__dict__.items()})

        dic_str = dic_str.replace('\'', "")

        driver.execute_query(f"""

            MATCH
                (entity_{entity_id}:Entity {{ name : "{entity_id}" }} )
            CREATE
                (entity_{entity_id}) 
                -[:IS_COMPONENT {{role: 'test_role' }}]
                ->({cmp_nme}_{entity_id}:Component:{cmp_nme} {dic_str})
                                                    
        """)

        if cmp_nme in dict_t.keys():

            """ this has to be a function that maps behaviour """

            prop_func = dict_t[cmp_nme][0]

            prop_name = dict_t[cmp_nme][1]

            prop_func ( driver, entity_id, component.__dict__.get(prop_name) )


In [24]:
URI = "neo4j://localhost"

AUTH = ("neo4j", "admin")

from neo4j import GraphDatabase

with GraphDatabase.driver(URI, auth=AUTH) as driver:

    driver.verify_connectivity()

    serialize_entity(driver, particle_parent)

    serialize_entity(driver, particle_instance)

This small resulting graph has the entity connected to each of its components. Additionally, the relationship with the parent is now presented with a bidirectional connection of the entities involved. It differs from the rendering engine, where to obtain this information, the Hierarchy component of the entity is requeried. This serialization introduces a direct relationship, enabling straightforward queries.

![Alt text](img-1.png)

We have created a graph equivalent of relational data. The serialization function can be extended with more relationships, although we aim to keep it simple and only connect the most important ones, such as the work done for the transform - hierarchy. A potential scenario would be the snapshot serialization of a running scene, where complex objects like particle systems with multiple instances interact with each other.

![Alt text](image-2.png)

Finally, based on the new layout of the data, we can ask queries such as:

- all the nodes with at least a parent that has geometry render component

This can be written in Cipher:

- MATCH p=(:Component:GeometryRenderer)--(:Entity)-[r:IS_CHILD]->() RETURN p

In [25]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:

    driver.verify_connectivity()

    r = driver.execute_query(f"""
                                                    
            MATCH p=(:Component:GeometryRenderer)--(:Entity)-[r:IS_CHILD]->() RETURN p
                                                    
    """)

    print (r)

EagerResult(records=[<Record p=<Path start=<Node element_id='76' labels=frozenset({'Component', 'GeometryRenderer'}) properties={'shader': 'None', 'geometry': 'None', 'color': '[255 255 255]'}> end=<Node element_id='71' labels=frozenset({'Entity'}) properties={'name': '1'}> size=2>>, <Record p=<Path start=<Node element_id='84' labels=frozenset({'Component', 'GeometryRenderer'}) properties={'shader': 'None', 'geometry': 'None', 'color': '[255 255 255]'}> end=<Node element_id='79' labels=frozenset({'Entity'}) properties={'name': '3'}> size=2>>], summary=<neo4j._work.summary.ResultSummary object at 0x7d401dd81300>, keys=['p'])


![Alt text](img-3.png)